In [1]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import descartes
from shapely.geometry import Point, Polygon
%matplotlib inline

In [2]:
uni18_df = pd.read_csv("ef2018cp.csv")

In [3]:
uni18_df

,UNITID,EFCIPLEV,CIPCODE,LINE,SECTION,LSTUDY,XEFTOTLT,EFTOTLT,XEFTOTLM,EFTOTLM,...,XEFUNKNM,EFUNKNM,XEFUNKNW,EFUNKNW,XEFNRALT,EFNRALT,XEFNRALM,EFNRALM,XEFNRALW,EFNRALW
0,100654,101,13.0,29,3,1,R,332,R,84,...,R,4,R,15,R,0,R,0,R,0
1,100654,102,13.0,99,3,2,R,260,R,68,...,R,3,R,10,R,0,R,0,R,0
2,100654,103,13.0,99,3,3,R,260,R,68,...,R,3,R,10,R,0,R,0,R,0
3,100654,104,13.0,99,3,4,R,63,R,22,...,R,2,R,2,R,0,R,0,R,0
4,100654,105,13.0,99,3,5,R,197,R,46,...,R,1,R,8,R,0,R,0,R,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164528,492962,643,52.0,20,2,43,R,913,R,389,...,R,48,R,29,R,3,R,1,R,2
164529,492962,644,52.0,15,2,44,R,120,R,42,...,R,7,R,6,Z,0,Z,0,Z,0
164530,492962,645,52.0,99,2,45,R,793,R,347,...,R,41,R,23,R,3,R,1,R,2
164531,492962,659,52.0,16,2,59,R,124,R,70,...,R,5,R,3,Z,0,Z,0,Z,0


In [4]:
uni18_df1 = uni18_df[['UNITID', 'EFCIPLEV', 'EFTOTLT', 'EFTOTLM', 'EFTOTLW']]

In [5]:
uni18_df1.head()

,UNITID,EFCIPLEV,EFTOTLT,EFTOTLM,EFTOTLW
0,100654,101,332,84,248
1,100654,102,260,68,192
2,100654,103,260,68,192
3,100654,104,63,22,41
4,100654,105,197,46,151


In [6]:
uni18_df1.groupby('EFCIPLEV')['EFTOTLW'].sum()

EFCIPLEV
101    707228
102    353340
103    347725
104     50374
105    297351
        ...  
836     12963
856        15
916     43879
936     43129
956       750
Name: EFTOTLW, Length: 171, dtype: int64

In [7]:
uni18_df2 = uni18_df1.loc[uni18_df1["EFCIPLEV"].isin([202])]

In [8]:
uni18_df2

,UNITID,EFCIPLEV,EFTOTLT,EFTOTLM,EFTOTLW
24,100654,202,603,462,141
161,100663,202,537,374,163
329,100706,202,2530,2022,508
468,100724,202,46,17,29
601,100751,202,4853,3678,1175
...,...,...,...,...,...
162572,484473,202,2,0,2
163289,486840,202,3857,3319,538
163429,486901,202,63,54,9
164169,490805,202,712,594,118


In [9]:
uni18_df3 = uni18_df2[['UNITID','EFTOTLT', 'EFTOTLM', 'EFTOTLW']]

In [10]:
uni18_df4 = uni18_df3.groupby(['UNITID']).sum()

In [11]:
uni18_df4

,EFTOTLT,EFTOTLM,EFTOTLW
UNITID,,,
100654,603,462,141
100663,537,374,163
100706,2530,2022,508
100724,46,17,29
100751,4853,3678,1175
...,...,...,...
484473,2,0,2
486840,3857,3319,538
486901,63,54,9


In [12]:
uni18_df5 = uni18_df4.reset_index()
uni18_df5

,UNITID,EFTOTLT,EFTOTLM,EFTOTLW
0,100654,603,462,141
1,100663,537,374,163
2,100706,2530,2022,508
3,100724,46,17,29
4,100751,4853,3678,1175
...,...,...,...,...
736,484473,2,0,2
737,486840,3857,3319,538
738,486901,63,54,9
739,490805,712,594,118


In [13]:
uni18_df5['W/M_Ratio'] = uni18_df5['EFTOTLW']/uni18_df5['EFTOTLM']
uni18_df5

,UNITID,EFTOTLT,EFTOTLM,EFTOTLW,W/M_Ratio
0,100654,603,462,141,0.305195
1,100663,537,374,163,0.435829
2,100706,2530,2022,508,0.251236
3,100724,46,17,29,1.705882
4,100751,4853,3678,1175,0.319467
...,...,...,...,...,...
736,484473,2,0,2,inf
737,486840,3857,3319,538,0.162097
738,486901,63,54,9,0.166667
739,490805,712,594,118,0.198653


# Bring in University Location Data

In [14]:
UniLoc = pd.read_csv("UniversityID_Lat_Long_ONLY.csv")

In [15]:
UniLoc

,UNITID,INSTNM,CITY,STABBR,ZIP,LATITUDE,LONGITUDE
0,100654,Alabama A & M University,Normal,AL,35762,34.783368,-86.568502
1,100663,University of Alabama at Birmingham,Birmingham,AL,35294-0110,33.505697,-86.799345
2,100690,Amridge University,Montgomery,AL,36117-3553,32.362609,-86.174010
3,100706,University of Alabama in Huntsville,Huntsville,AL,35899,34.724557,-86.640449
4,100724,Alabama State University,Montgomery,AL,36104-0271,32.364317,-86.295677
...,...,...,...,...,...,...,...
6689,48511113,Georgia Military College - Eastman,Eastman,GA,310236109,NaN,NaN
6690,48616901,American College of Barbering - Florence,Florence,KY,410421801,NaN,NaN
6691,49005401,HCI College - Fort Lauderdale Campus,Fort Lauderdale,FL,33309,NaN,NaN
6692,49146401,ABC Adult School - Cabrillo Lane,Cerritos,CA,907037748,NaN,NaN


JOIN!

In [16]:
df2018 = pd.merge(uni18_df5, UniLoc, on = "UNITID")
df2018

,UNITID,EFTOTLT,EFTOTLM,EFTOTLW,W/M_Ratio,INSTNM,CITY,STABBR,ZIP,LATITUDE,LONGITUDE
0,100654,603,462,141,0.305195,Alabama A & M University,Normal,AL,35762,34.783368,-86.568502
1,100663,537,374,163,0.435829,University of Alabama at Birmingham,Birmingham,AL,35294-0110,33.505697,-86.799345
2,100706,2530,2022,508,0.251236,University of Alabama in Huntsville,Huntsville,AL,35899,34.724557,-86.640449
3,100724,46,17,29,1.705882,Alabama State University,Montgomery,AL,36104-0271,32.364317,-86.295677
4,100751,4853,3678,1175,0.319467,The University of Alabama,Tuscaloosa,AL,35487-0100,33.211875,-87.545978
...,...,...,...,...,...,...,...,...,...,...,...
736,484473,2,0,2,inf,University of Florida-Online,Gainesville,FL,32611,29.649413,-82.339848
737,486840,3857,3319,538,0.162097,Kennesaw State University,Kennesaw,GA,30144,34.038807,-84.583052
738,486901,63,54,9,0.166667,Milligan University,Milligan,TN,37682,36.300147,-82.294070
739,490805,712,594,118,0.198653,Purdue University Northwest,Hammond,IN,46323,41.584324,-87.474236


Change to GEO Dataframe

In [17]:
crs = {'init':'epsg:4326'}
geometry = [Point(xy) for xy in zip(df2018["LONGITUDE"], df2018["LATITUDE"])]
geometry[:3]
geo_df2018 = gpd.GeoDataFrame(df2018, crs = crs, geometry = geometry)
geo_df2018.head()

C:\tools\Anaconda3\envs\GeoSpacial\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,UNITID,EFTOTLT,EFTOTLM,EFTOTLW,W/M_Ratio,INSTNM,CITY,STABBR,ZIP,LATITUDE,LONGITUDE,geometry
0,100654,603,462,141,0.305195,Alabama A & M University,Normal,AL,35762,34.783368,-86.568502,POINT (-86.56850 34.78337)
1,100663,537,374,163,0.435829,University of Alabama at Birmingham,Birmingham,AL,35294-0110,33.505697,-86.799345,POINT (-86.79935 33.50570)
2,100706,2530,2022,508,0.251236,University of Alabama in Huntsville,Huntsville,AL,35899,34.724557,-86.640449,POINT (-86.64045 34.72456)
3,100724,46,17,29,1.705882,Alabama State University,Montgomery,AL,36104-0271,32.364317,-86.295677,POINT (-86.29568 32.36432)
4,100751,4853,3678,1175,0.319467,The University of Alabama,Tuscaloosa,AL,35487-0100,33.211875,-87.545978,POINT (-87.54598 33.21187)


In [18]:
geo_df2018.to_csv('geo_df2018.csv', index = False)